# 匯入套件
* speech_recognition-->串聯google語言辨識api,將語音轉成文字,需要連網
* gTTS-->將文字轉成語音
* pygame.mixer-->是一個用來製造即撥放聲音的模組,稱為混音器
* tempfile-->主要用來創建暫存檔案和目錄,用完後會自動刪除
* jieba-->由百度開發的斷詞語料庫,裡面有兩萬多條詞
* time-->python提供time模塊格式化時間

In [1]:
import speech_recognition as sr
from gtts import gTTS
from pygame import mixer
import tempfile
import jieba
import time

pygame 2.2.0 (SDL 2.0.22, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


## 設定判斷語料

In [2]:
#list 用列表(list)定義美食分類
eat_list=["台菜","越式","義大利","法式"]
          
#list 用列表(list)定義訂位人數
quantity_list=["一","二","三","四","五","六","七","八","九","十","1","2","3","4","5","6","7","8","9","10"]

#設定空字串將美食行程加進去
item_list=[]
info={}

## 撰寫類別(class)內部功能

In [3]:
#定義類別名稱bot,提供主程式呼叫
class Bot():
    
    #使用class創建的物件後被執行,用於初始化
    def __init__(self):
        self.check_out=False
    #定義發聲方法
    def speak(self,sentence):
        with tempfile.NamedTemporaryFile(delete=True) as fp:
            tts=gTTS(text=sentence,lang='zh-tw')
            tts.save("{}.mp3".format(fp.name))
            mixer.music.load('{}.mp3'.format(fp.name))
            mixer.music.play()
            while mixer.music.get_busy():
                pass
    #定義監聽方法    
    def listen(self):
        global item_list
        r=sr.Recognizer()
        with sr.Microphone(device_index=0) as source:
            print("bot listening...")
            audio=r.listen(source)
            print("bot processing...")
        try:
            customer_schedule=r.recognize_google(audio,language='zh-TW')
            print("Google Speech Recognition think you said"+customer_schedule)
            if '結束' in customer_schedule:
                if not item_list:
                    self.speak("您尚未安排行程,請繼續安排")
                    time.sleep(2)
                else:
                    print("結束")
                    global check_out
                    self.check_out=True
                    
            elif "重新安排" in customer_schedule:
                item_list=[]
                self.speak("行程記錄已刪除,請重新安排行程")
                time.sleep(2)
            else:
                seg_list=jieba.cut(customer_schedule,cut_all=False)
                print(','.join(seg_list))
                print(customer_schedule)
                print("info:")
                print(info)
                
                for quantity in quantity_list:
                    if quantity in customer_schedule:
                        info['人數']=quantity
                        break
                name_counts=len(eat_list) #餐廳種類
                for name in eat_list:
                    if name_counts==0:
                        self.speak("目前無提供該項活動")
                    elif name in customer_schedule:
                        if name=="義大利":
                            info['美食']="蘇活義大利麵"
                        elif name=="台式" or "台":
                            info['美食']="石坊小井"
                        elif name=="法式":
                            info['美食']="露米亞歐法餐廳"
                print("info add:")
                print(info)
                item_list.append(info)
                print("item_list")
                print(item_list)
                self.speak("你安排了{}，{}，你可說結束完成規劃"
                           .format(info.get("美食"),info.get("人數","一")))
                name_counts-=1
        except sr.UnknownValueError:
            print("Google Speech could not understand audio")
            self.listen()
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service,{0}".format(e))
    #定義結束程式方法
    def checkOut(self):
        if self.check_out==True:
            for info in item_list:
                self.speak("你安排了{},{}位".format(info.get('美食'),info.get('人數','一')))
                break
            self.speak("請稍等,資訊將寄送至你的簡訊")
            time.sleep(4)
            print(end)